# Задача 5. Про парсинг информации

Есть сайт https://nplus1.ru/ , новости за дату на нем можно увидеть на страницах типа https://nplus1.ru/news/2023/03/28

Используя парсер, посчитайте суммарную длину текстов статей (в количестве слов или в количестве символов, как вам будет удобнее), которые были на N+1 в списке новостей в Июле 2023 года (пройтись по всем датам июля, собрать тексты статей, посчитать длину в словах/символах).

Пользуйтесь любыми удобными вам библиотеками

In [156]:
import time
import requests

from tqdm import tqdm
from selenium import webdriver
from bs4 import BeautifulSoup

In [159]:
headers = { 'Accept-Language' : 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
            'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
}

def get_links(soup, domain):
    '''
    Ищет все ссылки на новости со страницы
    :return: список строк - ссылок
    '''
    links = []
    pages = soup.find('section',class_=r'mb-10').find_all('a', class_="n1_climb_4")

    for link in pages:
        try:
            links.append(link.get('href'))
        except:
            pass
    links = [domain + l if 'https' not in l else l for l in links]
    return links

def get_chars(link):
    '''
    Считает число символов в новости по ссылке
    :return: число символов
    '''
    mystr = requests.get(link,
                         headers=headers).text
    soup = BeautifulSoup(mystr)
    try:
        text = soup.find_all('div', class_="col-span-6")[5].text
        chars = len(''.join(text.splitlines()).strip().replace(' ',''))
    except:
        pass
    return chars

In [162]:
basis = 'https://nplus1.ru/news/2023/07/'
urls = [basis + str(x) for x in range(1, 32)]

links=[]
driver = webdriver.Chrome('../chromedriver')

# Собираем все ссылки на новости
for url in urls:
    driver.get(url)
    pageSource = driver.page_source

    soup = BeautifulSoup(pageSource)#.find('section', class_='mb-10')
    links.extend(get_links(soup, domain=url))
driver.quit()

# Считаем число символов в каждой новости
symbols = 0
for link in tqdm(links):
    symbols += get_chars(link)
    # time.sleep(0.1)

print(symbols)

100%|██████████| 219/219 [01:55<00:00,  1.90it/s]

657092
